In [1]:
import pandas as pd
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
from io import StringIO # Python3 use: from io import StringIO
import seaborn as sns

models = tf.keras.models  # like 'from tensorflow.keras import models' (PyCharm import issue workaround)
layers = tf.keras.layers  # like 'from tensorflow.keras import layers' (PyCharm import issue workaround)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

DATA = os.path.join(module_path, 'Data', 'Augmented Data')
RESULTS = os.path.join(module_path, 'Results', 'Thesis')
FIGURES = os.path.join(module_path, 'Figures', 'Thesis')
from Scripts import Data_Loader_Functions as dL
from Scripts import Model_Architectures as mA
from Scripts import Results_Evaluation as rE
from Scripts import Print_Functions as pF

In [13]:
GROUP_2_PATH = os.path.join(DATA, "group_2")
CENTRAL_MODELS = os.path.join(module_path, 'Pain', 'Centralized')
FEDERATED_MODELS = os.path.join(module_path, 'Pain', 'Federated')

In [14]:
results_path = os.path.join(module_path, 'Results')

In [15]:
BCDL_df = pd.read_csv(os.path.join(results_path, 'BCDL.csv')).drop(columns=['Unnamed: 0'])
CDL_df = pd.read_csv(os.path.join(results_path, 'CDL.csv')).drop(columns=['Unnamed: 0'])
FDL_df = pd.read_csv(os.path.join(results_path, 'FDL.csv')).drop(columns=['Unnamed: 0'])
LDL_df = pd.read_csv(os.path.join(results_path, 'LDL.csv')).drop(columns=['Unnamed: 0'])
PFDL_df = pd.read_csv(os.path.join(results_path, 'PFDL.csv')).drop(columns=['Unnamed: 0'])

In [16]:
session_0 = BCDL_df[BCDL_df['Session'] == 0]

In [17]:
FDL_df = pd.concat((FDL_df, session_0.rename(columns={'BCDL': 'FDL'})))
LDL_df = pd.concat((LDL_df, session_0.rename(columns={'BCDL': 'LDL'})))
PFDL_df = pd.concat((PFDL_df, session_0.rename(columns={'BCDL': 'PFDL'})))

In [18]:
concat_df = BCDL_df.merge(CDL_df, how='inner', on=['Subject ID', 'Session', 'Frame', 'True Label', 'Seed'])
concat_df = concat_df.merge(FDL_df, how='inner', on=['Subject ID', 'Session', 'Frame', 'True Label', 'Seed'])
concat_df = concat_df.merge(LDL_df, how='inner', on=['Subject ID', 'Session', 'Frame', 'True Label', 'Seed'])
concat_df = concat_df.merge(PFDL_df, how='inner', on=['Subject ID', 'Session', 'Frame', 'True Label', 'Seed'])

In [24]:
concat_df = concat_df[['Seed', 'Session', 'Subject ID', 'Frame', 'True Label', 'BCDL', 'CDL', 'FDL', 'LDL', 'PFDL']]

In [28]:
concat_df = concat_df.sort_values(['Seed', 'Session', 'Subject ID', 'Frame'])

In [30]:
concat_df.to_csv('Concatenated.csv')

# Evaluation

In [32]:
df = pd.read_csv('Concatenated.csv')
df['True Label'] = np.minimum(df['True Label'], 1)
columns = ['BCDL', 'CDL', 'FDL', 'LDL', 'PFDL']

In [33]:
res_columns = [col + '_pred' for col in columns]
df = pd.concat((df, (df[columns] > 0.5).astype(int).rename(columns=dict(zip(columns, res_columns)))), axis=1)

In [35]:
from sklearn.metrics.classification import confusion_matrix

In [74]:
conf_m = np.array([confusion_matrix(sd_df['True Label'], sd_df['BCDL_pred']) for _, sd_df in df.groupby('Seed')])

In [82]:
conf_m.mean(axis=0)

array([[13906.9,  2555.1],
       [ 1446.3,  1636.7]])